In [4]:
from sklearn.datasets import load_breast_cancer
!pip install optuna


from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import lightgbm as lgb
import xgboost as xgb
# sklearn
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

import sklearn.metrics

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set()

from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import operator as op 
from itertools import combinations

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [5]:
X, y = load_breast_cancer(return_X_y=True)
X.shape

(569, 30)

In [6]:

X, dy = load_breast_cancer(return_X_y=True)
DX =  pd.DataFrame(X) # copy of the data for later
X =  pd.DataFrame(X) # copy to destroy


def brute_force_feat(X):

  comb= list(combinations(X.columns,3))

  def operations(x,a,b):
    if (x == '+'): 
      d =  op.add(a,b) 
    if (x == '-'): 
      d =  op.sub(a,b) 
    if (x == '*'): 
      d =  op.mul(a,b)     
    if (x == '/'): # divide by 0 error
      d =  op.truediv(a,(b + 1e-20)) 
    return d

  print('Operator Features')
  for x in ['*','/','+','-']:
    for y in ['*','/','+','-']:
      for i in comb:
        a = X.iloc[:,i[0]].values
        b = X.iloc[:,i[1]].values
        c = X.iloc[:,i[2]].values
        d = operations(x,a,b)
        e = operations(y,d,c)
        X[f'{i[0]}{x}{i[1]}{y}{i[2]}'] = e
        X[f'{i[0]}{x}{i[1]}'] = d

  X = X.loc[:,~X.columns.duplicated()]

  print('Dimensionality Features')
  # DIMENSIONALITY FEATURES
  # 2 is usually a good size, however higher may result in less reward for time taken
  X_TSNE = TSNE(n_components=2).fit_transform(X)
  X_DBSCAN = DBSCAN(eps=3, min_samples=2).fit(X)
  X_PCA = PCA(n_components=2).fit_transform(X)
  # Depending on how large the dataset is increase or decrease n 
  X_KNN64, indices = NearestNeighbors(n_neighbors=64, algorithm='ball_tree').fit(X).kneighbors(X)
  X = np.c_[X, X_KNN64] # NUMPY
  X = np.c_[X, X_PCA]
  X = np.c_[X, X_DBSCAN.labels_]
  X = np.c_[X, X_TSNE]
  X = np.where(np.isnan(X), 0, X)


  # Selecting best 50 columns 
  from sklearn.ensemble import RandomForestClassifier
  # PANDAS
  # SX = X.sample(n=500) # Sample, etc

  # NUMPY
  print('No. Features:', np.size(X,1))
  np.random.seed(42)
  # idx = np.random.randint(np.size(X,0), size=500) # make this 50% of your data just so it runs faster
  # X = [idx,:]
  # dy = dy.values[idx]
  # print(np.any(np.isnan(X)))
  # print(np.all(np.isfinite(X)))
  X = np.nan_to_num(X.astype(np.float32)) # prevents too large values error

  # SX IS A SAMPLE OF X
  SX = X
  sy = dy # other methods if this gives error #.values.ravel() #np.array(dy)[idx.astype(int)] #dy[idx]
  
  model_rfi = RandomForestClassifier(n_jobs = -1, class_weight = 'balanced_subsample', random_state=42)

  model_rfi.fit(SX, sy)
  print('Done!')

  return X, DX, model_rfi

# CX is the new features data | DX is the original data  | Model_rfi contains the feature importance
CX, DX, model_rfi = brute_force_feat(X)

Operator Features
Dimensionality Features
No. Features: 66683
Done!


In [7]:
def scaler_fuc(scaler, f, CX, DX):

  # order by most important features and take the f most important features
  fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:f]
  CX = CX[:,fs_indices_rfi]
  X = np.c_[DX, CX]
  # delete duplicates
  _, unq_col_indices = np.unique(X,return_index=True,axis=1)
  X = X[:,unq_col_indices]

  train_x, valid_x, train_y, valid_y = train_test_split(X, dy,
                                              test_size=0.25, random_state = 123)
  if (scaler == 'minmax'):
    scaler = MinMaxScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'stand'):
    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'log'):
    transformer = FunctionTransformer(np.log1p, validate=True)
    train_x = transformer.transform(train_x)
    valid_x = transformer.transform(valid_x)
    # prevent log 0 error
    np.seterr(divide = 'ignore')  # invalid value encountered in log1p
    train_x = np.where(np.isneginf(train_x), 0, train_x)
    valid_x = np.where(np.isneginf(valid_x), 0, valid_x)
    train_x = np.where(np.isinf(train_x), 0, train_x)
    valid_x = np.where(np.isinf(valid_x), 0, valid_x)
    train_x = np.where(np.isnan(train_x), 0, train_x)
    valid_x = np.where(np.isnan(valid_x), 0, valid_x)

  # turning these train/tests into lgb/xgb datasets
  dtrain_gbm = lgb.Dataset(train_x, label=train_y)
  dvalid_gbm = lgb.Dataset(valid_x, label=valid_y)

  dtrain_xbg = xgb.DMatrix(train_x, label=train_y)
  dvalid_xbg = xgb.DMatrix(valid_x, label=valid_y)
  return train_x, valid_x, train_y, valid_y,  dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg


In [38]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import lightgbm as lgb
import xgboost as xgb

models = ['mlp','knn','svc','gp','gnb','qda','ada', 'et', 'rf', 'xgb', 'gbm'] # when you want to try all of them/ iteration: 1 
# models = ['mlp','knn','svc', 'gp', 'gnb', 'qda', 'ada', 'et', 'rf', 'xgb', 'gbm']
comb = combinations(models, 3) 
comb = list(comb)

In [86]:
import math 
math.ceil(len(dy)*0.25)

143

In [158]:

class Objective:

    def __init__(self):
        self.best_gbm = None
        self._gbm = None
        self.best_xgb = None
        self._xgb = None
        self.predictions = None
        self.fpredictions = None

    def __call__(self, trial):

        i = trial.suggest_int("combos", 0, (len(comb)-1))
        gbm_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        xgb_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        rf_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        et_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        ada_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        qda_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        gnb_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        gp_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        svc_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        knn_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
        mlp_preds = np.zeros((math.ceil(len(dy)*0.25),1), dtype=np.int)
 

        ###############################################################################
        #                               . MLPClassifier                               #
        ###############################################################################
        if any(x == 'mlp' for x in comb[i]):
          mlp_f = trial.suggest_int("mlp_features", 0, 20)
          mlp_scaler = trial.suggest_categorical("mlp_Scaler", ['minmax','stand','log'])
          mlp_a = trial.suggest_loguniform("mlp_alpha", 1e-8, 1.0)
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(mlp_scaler, mlp_f, CX, DX)
          mlp = MLPClassifier(random_state=1, alpha = mlp_a, max_iter=500).fit(train_x, train_y)
          mlp_preds = mlp.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                           . KNeighborsClassifier                            #
        ###############################################################################
        if any(x == 'knn' for x in comb[i]):
          knn_f = trial.suggest_int("knn_features", 0, 20)
          knn_scaler = trial.suggest_categorical("knn_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(knn_scaler, knn_f, CX, DX)
          nn = trial.suggest_int("knn_nn", 1, 35)
          kp = trial.suggest_int("knn_p", 1, 2)
          knn = KNeighborsClassifier(n_neighbors = nn, p = kp, n_jobs = -1).fit(train_x, train_y)
          knn_preds = knn.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                            . Support Vector C                               #
        ###############################################################################
        if any(x == 'svc' for x in comb[i]):
          svc_f = trial.suggest_int("svc_features", 0, 20)
          svc_scaler = trial.suggest_categorical("svc_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(svc_scaler, svc_f, CX, DX)
          svc_kern = trial.suggest_categorical("svc_kern", ['linear', 'poly', 'rbf', 'sigmoid'])
          svc_C = trial.suggest_loguniform("svc_C", 1e-8, 5)
          svc_d = trial.suggest_int("svc_deg", 2, 10)
          svc = SVC(gamma='auto', kernel = svc_kern, C = svc_C,
                    probability=True, random_state=0).fit(train_x, train_y)
          svc_preds = svc.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                 . GaussianProcess   +  Radial-Basis Function                #
        ###############################################################################
        if any(x == 'gp' for x in comb[i]):
          gp_f = trial.suggest_int("gp_features", 0, 20)
          gp_scaler = trial.suggest_categorical("gp_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(gp_scaler, gp_f, CX, DX)
          a_gp = trial.suggest_loguniform("gp_a", 0.001, 10)
          gp_kern= trial.suggest_int("gp_kern", 1, 10)
          gpkernel = a_gp * RBF(gp_kern)
          gp = GaussianProcessClassifier(kernel=gpkernel, random_state=0, n_jobs = -1).fit(train_x, train_y)
          gp_preds = gp.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                                . GaussianNB                                 #
        ###############################################################################
        if any(x == 'gnb' for x in comb[i]):
          gnb_f = trial.suggest_int("gnb_features", 0, 20)
          gnb_scaler = trial.suggest_categorical("gnb_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(gnb_scaler, gnb_f, CX, DX)
          gnb_vs = trial.suggest_loguniform("gnb_vs", 1e-20, 1)
          gnb = GaussianNB(var_smoothing=gnb_vs).fit(train_x, train_y)
          gnb_preds = gnb.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                    . Quadratic Discriminant Analysis                        #
        ###############################################################################
        if any(x == 'qda' for x in comb[i]):
          qda_f = trial.suggest_int("qda_features", 0, 20)
          qda_scaler = trial.suggest_categorical("qda_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(qda_scaler, qda_f, CX, DX)
          qda_rp = trial.suggest_loguniform("qda_rp", 1e-20, 1)
          qda = QuadraticDiscriminantAnalysis(reg_param=qda_rp).fit(train_x, train_y)
          qda_preds = qda.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                               . Ada Boost                                   #
        ###############################################################################
        if any(x == 'ada' for x in comb[i]):
          ada_f = trial.suggest_int("ada_features", 0, 20)
          ada_scaler = trial.suggest_categorical("ada_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(ada_scaler, ada_f, CX, DX)
          ada_ne = trial.suggest_int("ada_ne", 1, 500) #500
          ada_lr = trial.suggest_loguniform("ada_lr", 0.05, 1)
          ada_algo = trial.suggest_categorical("ada_algo", ['SAMME', 'SAMME.R'])
          ada = AdaBoostClassifier(n_estimators = ada_ne, learning_rate = ada_lr,
                                   algorithm = ada_algo, random_state=0).fit(train_x, train_y)
          ada_preds = ada.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                              . Extra Trees                                  #
        ###############################################################################
        if any(x == 'et' for x in comb[i]):
          et_f = trial.suggest_int("et_features", 0, 20)
          et_scaler = trial.suggest_categorical("et_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(et_scaler,et_f, CX, DX)
          et_md = trial.suggest_int("et_max_depth", 1, 100)
          et_ne = trial.suggest_int("et_ne", 1, 500) #1000
          et = ExtraTreesClassifier(max_depth=et_md, n_estimators = et_ne,
                                     random_state=0).fit(train_x, train_y)
          et_preds = et.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                             . Random Forest                                 #
        ###############################################################################
        if any(x == 'rf' for x in comb[i]):
          rf_f = trial.suggest_int("rf_features", 0, 20)
          rf_scaler = trial.suggest_categorical("rf_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rf_scaler,rf_f, CX, DX)
          rf_md = trial.suggest_int("rf_max_depth", 1, 100)
          rf_ne = trial.suggest_int("rf_ne", 1, 500) #1000
          rf_cw = trial.suggest_categorical("rf_cw", ['balanced', 'balanced_subsample'])
          rf = RandomForestClassifier(max_depth=rf_md, n_estimators = rf_ne, n_jobs = -1,
                                     class_weight = rf_cw, random_state=0).fit(train_x, train_y)
          rf_preds = rf.predict_proba(valid_x)[:,1]
        ###############################################################################
        #                                 . XGBoost                                   #
        ###############################################################################
        if any(x == 'xgb' for x in comb[i]): 
          xgb_f = trial.suggest_int("xgb_features", 0, 20)
          xgb_scaler = trial.suggest_categorical("xgb_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(xgb_scaler,xgb_f, CX, DX)
          xgb_param = {
              "silent": 1,
              "objective": "binary:logistic", # change for multiclass
              "eval_metric": "auc",
              # "num_class": 3, # change this up depending on multiclass | dont use with binary
              "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
              "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
              "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
          }
          if xgb_param["booster"] == "gbtree" or xgb_param["booster"] == "dart":
              xgb_param["max_depth"] = trial.suggest_int("max_depth", 1, 100)
              xgb_param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
              xgb_param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
              xgb_param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
          if xgb_param["booster"] == "dart":
              xgb_param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
              xgb_param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
              xgb_param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
              xgb_param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)
          # Add a callback for pruning.
          xgb_pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc" )
          xgb_ = xgb.train(xgb_param, dtrain_xbg, evals=[(dvalid_xbg, "validation")], verbose_eval=False, callbacks=[xgb_pruning_callback])
          xgb_preds = xgb_.predict(dvalid_xbg)
          self._xgb = xgb_
        ###############################################################################
        #                          . Light Gradient Boosting                          #
        ###############################################################################
        if any(x == 'gbm' for x in comb[i]):
          gbm_f = trial.suggest_int("gbm_features", 0, 20)
          gbm_scaler = trial.suggest_categorical("gbm_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(gbm_scaler,gbm_f, CX, DX)
          gbm_param = {
            'objective': 'binary', # change for multiclass
            'metric': 'auc',
              "verbosity": -1,
              "boosting_type": "gbdt",
              "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10), 
              "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10),
              "num_leaves": trial.suggest_int("num_leaves", 2, 256), 
              "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0), 
              "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
              "bagging_freq": trial.suggest_int("bagging_freq", 1, 7), 
              "min_child_samples": trial.suggest_int("min_child_samples", 2, 20), 
          }
          # Add a callback for pruning.
          gbm_pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
          gbm = lgb.train(gbm_param, dtrain_gbm, valid_sets=[dvalid_gbm], verbose_eval=False, callbacks=[gbm_pruning_callback])
          gbm_preds = gbm.predict(valid_x)
          self._gbm = gbm
        ###############################################################################
        #                            . Stacking Strategy                              #
        ###############################################################################

        # # strat = trial.suggest_int("strat", 0, 1)

        preds = (gbm_preds + xgb_preds + rf_preds + \
        qda_preds + ada_preds + et_preds + \
        gnb_preds  + gp_preds + \
        svc_preds + knn_preds + mlp_preds) / 3 
        preds = preds[:1][0]

        # if (strat == 1): # Making an equation
        # a = trial.suggest_uniform("a", 0.1, 0.9)
        # b = trial.suggest_uniform("b", 0.1, 0.9)
        # c = trial.suggest_uniform("c", 0.1, 0.9)
        # d = trial.suggest_uniform("d", 0.1, 0.9)

        # def custom_sort(t):
        #   return t[0]

        # R = [[gbm_rmse,gbm_preds] , [xgb_rmse,xgb_preds], [rf_rmse, rf_preds], \
        #     [las_rmse,las_preds], [ada_rmse,ada_preds],[et_rmse,et_preds],\
        #     [lreg_rmse,lreg_preds], [rid_rmse,rid_preds]]
        # R.sort(key=custom_sort) # low to high
        # m1 = R[0][1]
        # m2 = R[1][1]
        # m3 = R[2][1]
        # # Using list comprehension. This may result in a divide by 0 error when rounding occurs.
        # # A fix would be to use a more precise library
        # preds = [ (a*((x**b) + (y**c))) + (d*z) for x,y,z in zip(m1,m2,m3) ]
        self.predictions = preds
        auc = roc_auc_score(valid_y, preds)
        return auc

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_gbm = self._gbm
            self.best_xgb = self._xgb
            self.fpredictions = self.predictions

In [159]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) # for log error

import optuna
objective = Objective()

# Setting SEED 
from optuna.samplers import TPESampler
sampler = TPESampler(seed=10)

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize",
    sampler=sampler
)
study.optimize(objective, n_trials=1000, callbacks=[objective.callback]) # change this to 500 + 

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_gbm = objective.best_gbm
best_xgb = objective.best_xgb

('mlp', 'svc', 'gp')


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

[I 2020-07-13 12:02:33,763] Finished trial#0 with value: 0.9925093632958801 with parameters: {'combos': 9, 'mlp_features': 4, 'mlp_Scaler': 'minmax', 'mlp_alpha': 3.500415365070801e-05, 'svc_features': 16, 'svc_Scaler': 'stand', 'svc_kern': 'poly', 'svc_C': 0.014769956099219564, 'svc_deg': 2, 'gp_features': 10, 'gp_Scaler': 'minmax', 'gp_a': 0.003994951112183678, 'gp_kern': 4}. Best is trial#0 with value: 0.9925093632958801.


('mlp', 'svc', 'gbm')


[I 2020-07-13 12:02:35,064] Finished trial#1 with value: 0.996046608406159 with parameters: {'combos': 16, 'mlp_features': 4, 'mlp_Scaler': 'log', 'mlp_alpha': 0.13382345725145836, 'svc_features': 1, 'svc_Scaler': 'minmax', 'svc_kern': 'linear', 'svc_C': 3.4595623493054765e-06, 'svc_deg': 5, 'gbm_features': 13, 'gbm_Scaler': 'stand', 'lambda_l1': 0.0007636587145416643, 'lambda_l2': 1.9034015928564192e-07, 'num_leaves': 143, 'feature_fraction': 0.7503408193922055, 'bagging_fraction': 0.509578861781209, 'bagging_freq': 2, 'min_child_samples': 14}. Best is trial#1 with value: 0.996046608406159.


('knn', 'gnb', 'gbm')


...

('knn', 'ada', 'rf')


[I 2020-07-13 12:29:51,824] Finished trial#998 with value: 0.9968789013732835 with parameters: {'combos': 72, 'knn_features': 9, 'knn_Scaler': 'stand', 'knn_nn': 1, 'knn_p': 1, 'ada_features': 7, 'ada_Scaler': 'log', 'ada_ne': 404, 'ada_lr': 0.7514405060695403, 'ada_algo': 'SAMME', 'rf_features': 5, 'rf_Scaler': 'log', 'rf_max_depth': 11, 'rf_ne': 182, 'rf_cw': 'balanced_subsample'}. Best is trial#561 with value: 0.9989596337910944.


('knn', 'rf', 'gbm')


[I 2020-07-13 12:29:53,904] Finished trial#999 with value: 0.9883478984602581 with parameters: {'combos': 79, 'knn_features': 10, 'knn_Scaler': 'stand', 'knn_nn': 3, 'knn_p': 1, 'rf_features': 2, 'rf_Scaler': 'minmax', 'rf_max_depth': 35, 'rf_ne': 457, 'rf_cw': 'balanced', 'gbm_features': 7, 'gbm_Scaler': 'stand', 'lambda_l1': 0.0003418604499651074, 'lambda_l2': 7.700217238627016e-07, 'num_leaves': 111, 'feature_fraction': 0.9433450712332289, 'bagging_fraction': 0.9548574089325709, 'bagging_freq': 3, 'min_child_samples': 9}. Best is trial#561 with value: 0.9989596337910944.


Best trial:
  Params: 
    combos: 73
    knn_features: 8
    knn_Scaler: stand
    knn_nn: 2
    knn_p: 1
    ada_features: 13
    ada_Scaler: log
    ada_ne: 437
    ada_lr: 0.495786247302282
    ada_algo: SAMME
    xgb_features: 6
    xgb_Scaler: stand
    booster: gbtree
    lambda: 0.4860408099326929
    alpha: 0.9976687514437015
    max_depth: 21
    eta: 5.4597174889339245e-06
    gamma: 0.002904350728501347
    grow_policy: depthwise


In [161]:
comb[73]

('knn', 'ada', 'xgb')

**BREAST CANCER**

0.9989 ('knn', 'ada', 'xgb')
```
    knn_features: 8
    knn_Scaler: stand
    knn_nn: 2
    knn_p: 1
    ada_features: 13
    ada_Scaler: log
    ada_ne: 437
    ada_lr: 0.495786247302282
    ada_algo: SAMME
    xgb_features: 6
    xgb_Scaler: stand
    booster: gbtree
    lambda: 0.4860408099326929
    alpha: 0.9976687514437015
    max_depth: 21
    eta: 5.4597174889339245e-06
    gamma: 0.002904350728501347
    grow_policy: depthwise
```